In [ ]:
from fastai2.basics import *
from fastai2.text.all import *
from fastai2.callback.all import *

In [ ]:
from nbdev.showdoc import *

In [ ]:
# all_slow

# Transfer learning in text

> How to fine-tune a language model and train a classifier

## Finetune a pretrained Language Model

First we get our data and tokenize it.

In [ ]:
path = untar_data(URLs.IMDB_SAMPLE)
df = pd.read_csv(path/'texts.csv')
df_tok,count = tokenize_df(df, 'text')

Then we put it in a `DataSource`. For a language model, we don't have targets, so there is only one transform to numericalize the texts. Note that `tokenize_df` returns the count of the words in the corpus to make it easy to create a vocabulary.

In [ ]:
splits = RandomSplitter()(range_of(df_tok))
vocab = make_vocab(count)
dsrc = DataSource(df_tok, [[attrgetter("text"), Numericalize(vocab)]], splits=splits, dl_type=LMDataLoader)

Then we use that `DataSource` to create a `DataBunch`. Here the class of `TfmdDL` we need to use is `LMDataLoader` which will concatenate all the texts in a source (with a shuffle at each epoch for the training set), split it in `bs` chunks then read continuously through it.

In [ ]:
dbunch = dsrc.databunch(bs=64, seq_len=72, after_batch=Cuda)

Or more simply with a factory method:

In [ ]:
dbunch = TextDataBunch.from_df(df_tok, text_col="text", text_vocab=vocab, is_lm=True)

In [ ]:
dbunch.show_batch()

,text,text_
0,"xxbos xxmaj the line is funnier in xxmaj england , where , away from xxmaj vixen ! 's native xxmaj america , the word "" xxunk "" has a whole new meaning . xxmaj sadly , it 's the only laugh you 'll get in this terrible sex comedy that is neither sexy nor funny . \n\n xxmaj oddly xxunk with painted - on xxunk , xxmaj xxunk xxmaj gavin ( acting","xxmaj the line is funnier in xxmaj england , where , away from xxmaj vixen ! 's native xxmaj america , the word "" xxunk "" has a whole new meaning . xxmaj sadly , it 's the only laugh you 'll get in this terrible sex comedy that is neither sexy nor funny . \n\n xxmaj oddly xxunk with painted - on xxunk , xxmaj xxunk xxmaj gavin ( acting ability"
1,"character , but the rest of his hard - xxunk xxunk want the hit - man dead . xxmaj like most crime thrillers in the 1960s and 1970s , "" big xxmaj guns "" xxunk to the cinematic morality that crime does not pay . xxmaj interestingly , the one man who has nothing to do with the murder of the wife and son of the hero survives while another xxunk the",", but the rest of his hard - xxunk xxunk want the hit - man dead . xxmaj like most crime thrillers in the 1960s and 1970s , "" big xxmaj guns "" xxunk to the cinematic morality that crime does not pay . xxmaj interestingly , the one man who has nothing to do with the murder of the wife and son of the hero survives while another xxunk the hero"
2,"but xxunk xxmaj mr . xxmaj burton forgets that his xxunk have working brains . xxmaj the over the top xxunk of xxmaj xxunk xxmaj xxunk xxmaj carter 's chimp , the xxunk of the humans ' xxunk , the ease of their escape , their extraordinary skills of xxunk ( this is an astronaut and a group of human xxunk suddenly riding full xxunk ) , the massive and immediate human","xxunk xxmaj mr . xxmaj burton forgets that his xxunk have working brains . xxmaj the over the top xxunk of xxmaj xxunk xxmaj xxunk xxmaj carter 's chimp , the xxunk of the humans ' xxunk , the ease of their escape , their extraordinary skills of xxunk ( this is an astronaut and a group of human xxunk suddenly riding full xxunk ) , the massive and immediate human xxunk"
3,"ingredients : it is just as poorly acted as their other efforts , we can watch xxmaj bo xxunk or xxunk for wet xxmaj t - shirt xxunk quite frequently , the story is just laughably idiotic , and the film takes itself much too seriously . xxmaj and then : xxmaj xxunk xxmaj xxunk in xxmaj africa ? \n\n xxmaj but it has a few things going for it . xxmaj",": it is just as poorly acted as their other efforts , we can watch xxmaj bo xxunk or xxunk for wet xxmaj t - shirt xxunk quite frequently , the story is just laughably idiotic , and the film takes itself much too seriously . xxmaj and then : xxmaj xxunk xxmaj xxunk in xxmaj africa ? \n\n xxmaj but it has a few things going for it . xxmaj bo"
4,"married to director xxmaj michael xxmaj xxunk . \n\n xxmaj but for me , the most memorable and weird aspect of the film is the seemingly gay subplot -- sort of like a xxup xxunk xxup mountain from the 1930s ( and we thought this was a xxup new idea ) . xxmaj gary xxmaj cooper 's character was raised by two men who hate women and do everything they can through","to director xxmaj michael xxmaj xxunk . \n\n xxmaj but for me , the most memorable and weird aspect of the film is the seemingly gay subplot -- sort of like a xxup xxunk xxup mountain from the 1930s ( and we thought this was a xxup new idea ) . xxmaj gary xxmaj cooper 's character was raised by two men who hate women and do everything they can through much"
5,"males and xxunk some of their body parts , was probably the most xxunk serial killer of our generation . xxmaj however , the real sick individuals are the filmmakers of this awful spectacle who should have had their heads examined before deciding to xxunk this awful ` xxunk ' project . xxmaj this is not an easy film to xxunk , even though xxmaj jeffrey would have easily xxunk it with","and xxunk some of the

Then we have a convenience method to directly grab a `Learner` from it, using the `AWD_LSTM` architecture.

In [ ]:
learn = language_model_learner(dbunch, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, opt_func = partial(Adam, wd=0.1)).to_fp16()

In [ ]:
learn.freeze()
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.450261,4.002472,0.273968,54.733280,00:15


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(4, 1e-2, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.380655,4.388360,0.237928,80.508240,00:19
1,4.286285,4.307310,0.248634,74.240486,00:17
2,3.900547,4.226243,0.261510,68.459518,00:17
3,3.377858,4.264581,0.262330,71.135124,00:17


Once we have fine-tuned the pretrained language model to this corpus, we save the encoder since we will use it for the classifier.

In [ ]:
learn.show_results()

,input,target,pred
0,"xxbos xxmaj this is n't one of xxmaj arbuckle 's or xxmaj keaton 's better films , that 's for sure . xxmaj fatty 's wife is tired of all his heavy drinking , so she takes him to a xxunk where a psychiatrist ( keaton ) claims to have a guaranteed cure ! xxmaj well , once there , xxmaj arbuckle accidentally eats a xxunk and is taken to surgery .","xxmaj this is n't one of xxmaj arbuckle 's or xxmaj keaton 's better films , that 's for sure . xxmaj fatty 's wife is tired of all his heavy drinking , so she takes him to a xxunk where a psychiatrist ( keaton ) claims to have a guaranteed cure ! xxmaj well , once there , xxmaj arbuckle accidentally eats a xxunk and is taken to surgery . xxmaj","xxmaj this is a a of those hollywood 's better xxmaj keaton 's better films , but 's what me , xxmaj the xxmaj wife , a of his the xxunk drinking , but he 's a to the xxunk xxunk he man ( robert ) is that be a xxunk cure . xxmaj the , he he , he jigsaw is xxunk a xxunk and turns thrown to the . xxmaj"
1,"references to xxmaj asimov 's xxmaj laws of xxmaj xxunk , but they are an xxunk to the xxup cgi and action scenes . \n\n xxmaj smith is likable , as he is in most of his films , but honestly , the story is n't that good . you figure it out long before these genius characters do . xxbos xxmaj my husband and i bought the xxmaj old xxmaj school","to xxmaj asimov 's xxmaj laws of xxmaj xxunk , but they are an xxunk to the xxup cgi and action scenes . \n\n xxmaj smith is likable , as he is in most of his films , but honestly , the story is n't that good . you figure it out long before these genius characters do . xxbos xxmaj my husband and i bought the xxmaj old xxmaj school xxmaj","to the xxunk 's "" xxunk of xxmaj xxunk . and this did not xxunk of the xxunk xxunk . the scenes . xxmaj xxmaj the 's a as but he is a a of his films , but he , he movie is not that good . xxmaj can out out . ago you two xxunk will n't xxmaj xxmaj this wife and i bought this xxup xxunk xxmaj school xxmaj"
2,"is the central philosophy of post modern capitalism : your xxunk is the most important thing . you must place yourself in the center , and no one else but you is the matter . this is the philosophy the film stands for , and that 's just it 's first sin . \n\n xxmaj if you disagree with me on the political side , xxmaj i 'm sure you will agree","the central philosophy of post modern capitalism : your xxunk is the most important thing . you must place yourself in the center , and no one else but you is the matter . this is the philosophy the film stands for , and that 's just it 's first sin . \n\n xxmaj if you disagree with me on the political side , xxmaj i 'm sure you will agree that","n't only xxunk of the modern capitalism . xxmaj xxunk are xxunk only important thing of xxmaj can see your in a woods of and you one will . you can n't only of xxmaj is a only of movie has for . and it 's what a 's not rate . xxmaj xxmaj the you want with the , the xxmaj side , you i 'm not you will be with"
3,"could had been a fine or even classic xxmaj b - movie , but here it is just a bad attempt at film - making . \n\n xxmaj director xxmaj philip xxup j. xxmaj jones tells the tale of xxmaj james xxmaj bishop ( matt xxmaj xxunk ) , a young xxunk resident , who just got his dream job at xxmaj st . xxmaj andrew xxmaj mental xxmaj hospital ; but","had been a fine or even classic xxmaj b - movie , but here it is just a bad attempt at film - making . \n\n xxmaj director xxmaj philip xxup j. xxmaj jones tells the tale of xxmaj james xxmaj bishop ( matt xxmaj xxunk ) , a young xxunk resident , who just got his dream job at xxmaj st . xxmaj andrew xxmaj mental xxmaj hospital ; but the","n't been a bit xxunk even xxunk movie hollywood - movie . but it 's was n't a xxunk movie at xxunk - making . xxmaj xxmaj the xxmaj xxunk xxmaj j. xxmaj xxunk is the story of a louis xxmaj xxunk ( james xxmaj damon ) , a professional man who who who is has his name job in a xxunk . x

In [ ]:
learn.save_encoder('enc1')

## Use it to train a classifier

For classification, we need to use two set of transforms: one to numericalize the texts and the other to encode the labels as categories.

In [ ]:
splits = RandomSplitter()(range_of(df_tok))
dsrc = DataSource(df_tok, splits=splits, tfms=[
    [attrgetter("text"), Numericalize(vocab)],
    [attrgetter("label"), Categorize()]], dl_type=SortedDL)

We opnce again use a subclass of `TfmdDL` for the dataloaders, since we want to sort the texts (sortish for the training set) by order of lengths. We also use `pad_collate` to create batches form texts of different lengths.

In [ ]:
dbunch = dsrc.databunch(before_batch=pad_input, after_batch=Cuda)

In [ ]:
dbunch.show_batch(max_n=2)

,text,category
0,"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n\n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , xxunk bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj victor xxmaj vargas became i was always aware that something did n't quite feel right . xxmaj victor xxmaj vargas suffers from a certain xxunk on the director 's part . xxmaj apparently , the director thought that the ethnic backdrop of a xxmaj latino family on the lower east side , and an xxunk storyline would make the film critic proof . xxmaj he was right , but it did n't fool me . xxmaj raising xxmaj victor xxmaj vargas is",negative
1,"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the xxunk and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with the xxunk possible scenarios to get the two protagonists together in the end . xxmaj in fact , all its charm is xxunk , contained within the characters and the setting and the plot … which is highly believable to xxunk . xxmaj it 's easy to think that such a love story , as beautiful as any other ever told , * could * happen to you … a feeling you do n't often get from other romantic comedies",positive


Or with a factory method of `TextDataBunch`:

In [ ]:
dbunch = TextDataBunch.from_df(df_tok, text_col="text", text_vocab=vocab, label_col='label')

Then we once again have a convenience function to create a classifier from this `DataBunch` with the `AWD_LSTM` architecture.

In [ ]:
learn = text_classifier_learner(dbunch, AWD_LSTM, metrics=[accuracy], path=path, opt_func=Adam, drop_mult=0.5)

In [ ]:
learn = learn.load_encoder('enc1')

Then we can train with gradual unfreezing and differential learning rates.

In [ ]:
learn.fit_one_cycle(4, moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.627347,0.444851,0.840000,00:10
1,0.564891,0.410152,0.825000,00:11
2,0.472728,0.367973,0.860000,00:10
3,0.429697,0.361784,0.845000,00:10


In [ ]:
learn.unfreeze()
learn.opt = learn.create_opt()
learn.fit_one_cycle(8, slice(1e-5,1e-3), moms=(0.8,0.7,0.8))

epoch,train_loss,valid_loss,accuracy,time
0,0.382893,0.351107,0.850000,00:17
1,0.396118,0.360789,0.845000,00:18
2,0.324209,0.432927,0.790000,00:16
3,0.287399,0.371246,0.815000,00:20
4,0.254755,0.375270,0.835000,00:19
5,0.238167,0.375107,0.850000,00:16
6,0.220819,0.405301,0.825000,00:16
7,0.215843,0.400871,0.835000,00:17


In [ ]:
learn.show_results(max_n=5)

,text,category,category_
0,"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n\n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , xxunk bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj victor xxmaj vargas became i was always aware that something did n't quite feel right . xxmaj victor xxmaj vargas suffers from a certain xxunk on the director 's part . xxmaj apparently , the director thought that the ethnic backdrop of a xxmaj latino family on the lower east side , and an xxunk storyline would make the film critic proof . xxmaj he was right , but it did n't fool me . xxmaj raising xxmaj victor xxmaj vargas is",negative,negative
1,"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the xxunk and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with the xxunk possible scenarios to get the two protagonists together in the end . xxmaj in fact , all its charm is xxunk , contained within the characters and the setting and the plot … which is highly believable to xxunk . xxmaj it 's easy to think that such a love story , as beautiful as any other ever told , * could * happen to you … a feeling you do n't often get from other romantic comedies",positive,positive
2,"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj xxunk , after xxunk ) , i can xxunk join both xxunk of "" at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n\n xxmaj it 's usually satisfying to watch a film director change his style / subject , but xxmaj soderbergh 's most recent stinker , xxmaj the xxmaj girlfriend xxmaj xxunk ) , was also missing a story , so narrative ( and editing ? ) seem to suddenly be xxmaj soderbergh 's main challenge . xxmaj strange , after xxunk years in the business . xxmaj he was probably never much good at narrative , just xxunk it well inside "" edgy "" projects . \n\n xxmaj none of this excuses him this present , almost diabolical",negative,negative
3,"xxbos xxmaj this film sat on my xxmaj xxunk for weeks before i watched it . i xxunk a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj xxunk . \n\n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" la xxmaj xxunk , "" based on a play by xxmaj arthur xxmaj xxunk , who is given an "" inspired by "" credit . xxmaj it starts from one person , a prostitute , standing on a street corner in xxmaj brooklyn . xxmaj she is picked up by a home contractor , who has sex with her on the hood of a car , but ca n't come . xxmaj he refuses to pay her . xxmaj when he 's off xxunk , she",positive,positive
4,"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first 3d game , or even the first xxunk - up . xxmaj it 's also one of the first xxunk games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - xxunk gaming experience in general . xxmaj with graphics that are terribly dated today , the game xxunk you into the role of xxunk even * think * xxmaj i 'm going to attempt spelling his last name ! ) , an xxmaj american xxup xxunk . caught in an underground bunker . xxmaj you fight and search your way through xxunk in order to achieve different xxunk for the six xxunk , let 's face it , most of them are just an excuse to hand you a weapon",positive,positive


In [ ]:
from fastai2.interpret import *

In [ ]:
interp = Interpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(6)

,input,target,predicted,probability,loss
0,"xxbos xxmaj over xxmaj her xxmaj dead xxmaj body was a nice little xxunk was decent and entertaining , while still being pretty xxunk were a few cliché 's , but i found most stuff xxunk first i did n't think it was going to be good at all , when it started xxunk you can get past the first 20 minutes though , the movie starts getting more xxunk film was n't burst out in laughter hilarious , and was n't xxup oh xxup my xxup xxunk xxunk was just a movie that you can sit down and enjoy for how enjoyable it xxunk do n't see how this movie was xxunk rating is just a bit too xxunk could 've dealt with a xxunk a xxunk , giving this movie a 1 is xxunk was pretty good , and there was nothing horrible enough about it to give",positive,negative,0.9911274313926697,4.724791526794434
1,"xxbos xxmaj not sure if it was right or wrong , but i read thru the other comments before watching the xxunk have to say i disagree with most of the negative comments or problems people have had with it . \n\n xxmaj as a first time "" lone xxmaj wolf "" director / producer , i like to see things that i can xxunk to , not necessarily from the pro 's , but by people just getting their feet wet like me . \n\n xxmaj if indeed this is also from a first - xxunk , as i read , i applaud the xxunk job then in that respect ! xxmaj there were some comments about the xxunk thought it was quite nice for the xxunk say it kind of xxunk along for a while , but i found that created tension xxunk being xxunk of it ,",positive,negative,0.989062488079071,4.515562057495117
2,"xxbos xxmaj of life in ( some ) colleges . xxmaj of course there were artistic xxunk taken , but some of what you saw in this film go on in some colleges . \n\n i went to colleges in xxmaj southern xxmaj california where the xxunk pretty much hang around with their own . xxmaj it 's funny because these are schools that want racial xxunk , equality etc . and i can honestly say , that it 's there . xxmaj but the thing is when class lets out , or when they 're just hanging out waiting for class , they ( students ) seem to just hang around with people of their own race or ethnicity . xxmaj is that bad ? xxmaj not really . xxmaj everyone needs a feeling of belonging . xxmaj but like the school paper of one of the schools i",positive,negative,0.9781860709190369,3.8252058029174805
3,xxbos i ca nt help it but i seem to like films that are meant to be scary and are just plain bad . i have personally listed it in my own top 10 worst movies right under creatures of the xxunk xxmaj watch this film and have a laugh just do n't expect to see any academy awards for acting . xxmaj more chance of understanding the film its self . xxmaj in all honesty though i have seen much worse than this . xxmaj plus some xxunk xxunk round the desert xxunk the same people out that just died is that unbelievable that its got to be original . i think its one of those love or hate movies . you can make up your own mind yes its awful but it pulls it off somehow that s why i love it,negative,positive,0.9513773322105408,3.0236663818359375
4,"xxbos xxmaj like xxmaj tarzan the xxmaj ape xxmaj man ( 1932 ) , only more so . xxmaj there 's more of everything , more animals , more varied xxmaj african tribes , and scenes in which the thought must be , if this was good with three or four lions , forty would be better . xxmaj tarzan xxunk with xxunk the crocodile machine xxunk in the water like a rolling pin , around and around , jaws xxunk . xxmaj tarzan can kill it with his xxunk knife if the xxunk xxunk would hold still . xxmaj tarzan kills lions and xxunk and a xxunk xxunk number of animals . xxmaj his friends are real xxunk , people wearing larger ape costumes , and xxunk . xxmaj in fact , they use xxmaj indian xxunk more friendly and xxunk than xxmaj african xxunk costume ears attached to",positive,negative,0.9405072331428528,2.821899890899658
5,"xxbos xxmaj this is yet another bad movie that you should 